In [1]:
import sentencepiece as spm

In [ ]:
%cd /content/drive/MyDrive/zindi_opt/tokenizer

In [ ]:
spm.SentencePieceTrainer.train(
    input='/content/drive/MyDrive/zindi_opt/data/master.txt',
    model_prefix='combined',
    vocab_size=31999,  # Reduce vocab size by 1 to make room for pad token
    character_coverage=1.0,
    model_type='bpe',
    input_sentence_size=1000000,
    shuffle_input_sentence=True,
)

In [ ]:
sp = spm.SentencePieceProcessor()
sp.load('combined.model')

In [ ]:
vocab = [sp.id_to_piece(i) for i in range(sp.get_piece_size()) if not sp.is_control(i) and not sp.is_unknown(i)]

In [ ]:
vocab.append("<pad>")

In [ ]:
spm.SentencePieceTrainer.train(
    input='/content/drive/MyDrive/zindi_opt/data/master.txt',
    model_prefix='combined_with_pad',
    vocab_size=32000,
    model_type='bpe',
    character_coverage=1.0,
    input_sentence_size=1000000,
    shuffle_input_sentence=True,
    user_defined_symbols=['<pad>'],
    pad_id=31999,  # Set the pad_id to 31999 (the last index)
    unk_id=0,  # Ensure <unk> is at index 0
    bos_id=1,  #  <s> token
    eos_id=2,  #  </s> token
)

In [ ]:
sp_new = spm.SentencePieceProcessor()
sp_new.load('combined_with_pad.model')
print(sp_new.piece_to_id("<pad>"))  # Should print 31999
print(sp_new.get_piece_size())  # Should print 32000

In [ ]:
tokens = {}
counter = 0
with open("/content/drive/MyDrive/zindi_opt/tokenizer/combined.vocab", "r") as f:
    for line in f.readlines():
        tokens[str(line.split()[0])]=counter
        counter+=1
    tokens["<pad>"]=counter

In [ ]:
import json 
with open('combined_vocab_with_pad.json', 'w', encoding='utf-8') as f:
    json.dump(tokens, f, ensure_ascii=False, indent=2)

In [3]:
from transformers import MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained(
    "Helsinki-NLP/opus-mt-en-fr", 
    source_spm='/home/rana/Projects/zindi/tokenizer_custom/combined_pad/combined_with_pad.model',
    target_spm='/home/rana/Projects/zindi/tokenizer_custom/combined_pad/combined_with_pad.model',
    vocab='/home/rana/Projects/zindi/tokenizer_custom/combined_pad/combined_formatted_pad.json',
    source_lang='dyu',
    target_lang='fr'
)

/home/rana/Projects/zindi/transformers/src/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
tokenizer.save_pretrained('/home/rana/Projects/zindi/tokenizer_custom/combined_pad_V1')

('/home/rana/Projects/zindi/tokenizer_custom/combined_pad_V1/tokenizer_config.json',
 '/home/rana/Projects/zindi/tokenizer_custom/combined_pad_V1/special_tokens_map.json',
 '/home/rana/Projects/zindi/tokenizer_custom/combined_pad_V1/vocab.json',
 '/home/rana/Projects/zindi/tokenizer_custom/combined_pad_V1/source.spm',
 '/home/rana/Projects/zindi/tokenizer_custom/combined_pad_V1/target.spm',
 '/home/rana/Projects/zindi/tokenizer_custom/combined_pad_V1/added_tokens.json')